In [1]:
@file:DependsOn("com.londogard:nlp:1.2.0-BETA3")

In [2]:
import com.londogard.nlp.meachinelearning.predictors.sequence.HiddenMarkovModel
import org.jetbrains.kotlinx.multik.api.mk
import org.jetbrains.kotlinx.multik.api.ndarray

val text = """In	IN
an	DT
Oct.	NNP
19	CD
review	NN
of	IN
``	``
The	DT
Misanthrope	NN
''	''
at	IN
Chicago	NNP
's	POS
Goodman	NNP
Theatre	NNP
(	(
``	``
Revitalized	VBN
Classics	NNS
Take	VBP
the	DT
Stage	NN
in	IN
Windy	NNP
City	NNP"""
val (tokensText, tagsText) = text
    .split('\n')
    .map {
        val (a, b) = it.split('\t')
        a to b
    }.unzip()
val tokenMap = (tokensText).toSet().withIndex().associate { elem -> elem.value to elem.index }
val tagMap = (tagsText + "BOS").toSet().withIndex().associate { elem -> elem.value to elem.index }
val reversetagMap = tagMap.asIterable().associate { (key, value) -> value to key }
val hmm = HiddenMarkovModel(
    tagMap.asIterable().associate { (key, value) -> value to key },
    tokenMap.asIterable().associate { (key, value) -> value to key },
    BegginingOfSentence = tokenMap.getOrDefault("BOS", 0)
)

val x = listOf(mk.ndarray(tokensText.mapNotNull(tokenMap::get).toIntArray()))
val y = listOf(mk.ndarray(tagsText.mapNotNull(tagMap::get).toIntArray()))

hmm.fit(x, y)
hmm.predict(x)

[[0, 1, 2, 3, 4, 0, 5, 1, 4, 6, 0, 2, 7, 2, 2, 8, 5, 9, 10, 11, 1, 4, 0, 2, 2]]

In [3]:
y

[[0, 1, 2, 3, 4, 0, 5, 1, 4, 6, 0, 2, 7, 2, 2, 8, 5, 9, 10, 11, 1, 4, 0, 2, 2]]

In [4]:
hmm.predict(x).map { t -> t.data.map { reversetagMap[it] } }

[[IN, DT, NNP, CD, NN, IN, ``, DT, NN, '', IN, NNP, POS, NNP, NNP, (, ``, VBN, NNS, VBP, DT, NN, IN, NNP, NNP]]